# Figure out where and how precursor features are generated

1. Figure out where and how precursor features are **written**

In [111]:
import pandas as pd
import re 
import pysam
import numpy as np
import os

In [2]:
feature_df = pd.read_csv('output_dir/miRge.2022-04-26_12-04-16/feature_files/SRR772403_dataset_15_refined_features.csv')

So clearly 

See google slides as of June 1, June 2, 2022. We're way past this. 

In [4]:
def chunks(arr, n):
    return [arr[i:i+n] for i in range(0, len(arr), n)]

In [5]:
precusorStrFile = 'output_dir/miRge.2022-04-26_12-04-16/unmapped_tmp/SRR772403_precursor.str'

In [6]:
with open(precusorStrFile, 'r') as inf2:
    totalContent = inf2.readlines()
    totalContentSplit = chunks(totalContent, 3)
    for item in totalContentSplit:
        precusorName = item[0].strip()[1:]
        precusorSeq = item[1].strip()
        precusorStr = item[2].strip().split()[0]
        try:
            if len(item[2].strip().split()[1]) >= 3:
                mfeValue = float(item[2].strip().split()[1][1:-1].strip())
            else:
                mfeValue = float(item[2].strip().split()[2][:-1].strip())
        except:
            mfeValue = 0.0

In [7]:
precusorName

'SRR772403:miRCluster_206_23:chrX:66018937_66018959+:precusor_2'

In [8]:
precusorSeq

'GGACACUCCAUGUGGUAGAGUGUCAGUUUGUCAAAUACCCCAAGUGCGGCACAUGCUUACCAGCUCUAGGCCAGGGCAGAUGGGAUAUGACGAAUGGACUGCCAGCUGGA'

In [9]:
precusorStr

'......((((..(((((....(((.((((((((..((.((((..((((((....(((....))).....)))...)))..)))).)))))))))).))))))))..))))'

In [12]:
len(totalContentSplit)

184

In [15]:
item[0].strip()

'>SRR772403:miRCluster_206_23:chrX:66018937_66018959+:precusor_2'

In [17]:
help(str.strip)

Help on method_descriptor:

strip(self, chars=None, /)
    Return a copy of the string with leading and trailing whitespace removed.
    
    If chars is given and not None, remove characters in chars instead.



In [18]:
%%bash
head output_dir/miRge.2022-04-26_12-04-16/unmapped_tmp/SRR772403_precursor.str

>SRR772403:miRCluster_4_24:chr1:40754371_40754394+:precusor_1
UCUGUCACCUCCUUACUAGAGUAGGGUGUGCCUCACUGCGUCUCCGUUCUUUCUGGGCAGUCUUUGCUACUGUAAACAUCCUUGACUGGAAGCUGUAAGGUGUUCAGAG
((((.(((((..((((.(((((((((..(((((....(((....))).......)))))..))))))).))))))((((((......)))...))).)))))..)))). (-29.70)
>SRR772403:miRCluster_4_24:chr1:40754371_40754394+:precusor_2
UUCUGGGCAGUCUUUGCUACUGUAAACAUCCUUGACUGGAAGCUGUAAGGUGUUCAGAGGAGCUUUCAGUCGGAUGUUUACAGCGGCAGGCUGCCACGGUCGUCCCCAG
..(((((((((((..(((.((((((((((((..(((((((((((................))))))))))))))))))))))).))))))))))).))).......... (-54.10)
>SRR772403:miRCluster_5_22:chr1:40754413_40754434+:precusor_1
UCCGUUCUUUCUGGGCAGUCUUUGCUACUGUAAACAUCCUUGACUGGAAGCUGUAAGGUGUUCAGAGGAGCUUUCAGUCGGAUGUUUACAGCGGCAGGCUGCCACGGUCGU
..........(((((((((((..(((.((((((((((((..(((((((((((................))))))))))))))))))))))).))))))))))).))).... (-54.10)
>SRR772403:miRCluster_5_22:chr1:40754413_40754434+:precusor_2


In [17]:
premiRNASeq = "UCUGUCACCUCCUUACUAGAGUAGGGUGUGCCUCACUGCGUCUCCGUUCUUUCUGGGCAGUCUUUGCUACUGUAAACAUCCUUGACUGGAAGCUGUAAGGUGUUCAGAG"
premiRNAStructure = "((((.(((((..((((.(((((((((..(((((....(((....))).......)))))..))))))).))))))((((((......)))...))).)))))..))))."


In [2]:
import mirge.forgi.graph.bulge_graph as fgb

In [18]:
bg1 = fgb.BulgeGraph()
fa = '\n'.join(['>PrecusorStr', premiRNASeq, premiRNAStructure])
bg1.from_fasta(fa)

In [19]:
hairpinCount = 0
interiorLoopCount = 0
for string in bg1.to_bg_string().strip().split('\n'):
    if 'define' in string:
        contentTmp = string.split(' ')
        elementType = contentTmp[1]
        if elementType[0] == 'h':
            hairpinCount = hairpinCount + 1
        if elementType[0] == 'i':
            interiorLoopCount = interiorLoopCount + 1

In [16]:
print(bg1.to_bg_string())

name PrecusorStr
length 111
seq UCCGUUCUUUCUGGGCAGUCUUUGCUACUGUAAACAUCCUUGACUGGAAGCUGUAAGGUGUUCAGAGGAGCUUUCAGUCGGAUGUUUACAGCGGCAGGCUGCCACGGUCGU
seq_ids 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111
define f1 1 10
define s0 11 13 105 107
define s1 14 21 96 103
define i3 104 104
define i0 22 23
define s2 24 26 93 95
define i1 27 27 92 92
define s3 28 39 80 91
define i2 40 41
define s4 42 52 69 79
define h0 53 68
define t1 108 111
connect s0 f1 i3 t1
connect s1 i3 i0
connect s2 i1 i0
connect s3 i1 i2
connect s4 h0 i2



In [14]:
hairpinCount

1

In [ ]:
UCUGUCACCUCCUUACUAGAGUAGGGUGUGCCUCACUGCGUCUCCGUUCUUUCUGGGCAGUCUUUGCUACUGUAAACAUCCUUGACUGGAAGCUGUAAGGUGUUCAGAG
((((.(((((..((((.(((((((((..(((((....(((....))).......)))))..))))))).))))))((((((......)))...))).)))))..)))). (-29.70)

# Below I've pasted content of the featuresInPrecusor method

In [28]:
bg = fgb.BulgeGraph()
fa = '\n'.join(['>prunedPrecursorStr', premiRNASeq, premiRNAStructure])
bg.from_fasta(fa)
bindingCount1 = 0
bindingCount2 = 0
for item in premiRNAStructure:
    if item == '(' :
        bindingCount1 = bindingCount1 + 1
    if item == ')' :
        bindingCount2 = bindingCount2 + 1
# Calculate bindingCount in the CorePrecusor
bindingCount = max([bindingCount1, bindingCount2])
# Calculate the hairpinCount and interiorLoopCount
hairpinCount = 0
interiorLoopCount = 0
for string in bg.to_bg_string().strip().split('\n'):
    if 'define' in string:
        contentTmp = string.split(' ')
        elementType = contentTmp[1]
        if elementType[0] == 'h':
            hairpinCount = hairpinCount + 1
        if elementType[0] == 'i':
            interiorLoopCount = interiorLoopCount + 1
# Calculate apicalLoopSize of the CorePrecusor
# To get compatible with the older version of forgi, the below statement is abandoned.
#elementTypeSeq, indeSeq = bg.to_element_string(with_numbers=True).split('\n')
elementTypeSeq, indeSeq = to_element_string(bg, with_numbers=True).split('\n')

elementIndexList = [elementTypeSeq[i]+indeSeq[i] for i in range(len(elementTypeSeq))]
elementSeqList = []
elementlocationList = []
# the position is 0-based
for index, item in enumerate(elementIndexList):
    if len(elementlocationList) == 0:
        elementSeqList.append(item)
        elementlocationList.append([index, index])
    elif item != elementSeqList[-1]:
        elementSeqList.append(item)
        elementlocationList.append([index, index])
    else:
        elementlocationList[-1][1] = index
elementDic = {}
for i in range(len(elementSeqList)):
    if elementSeqList[i] not in elementDic.keys():
        elementDic.update({elementSeqList[i] : [elementlocationList[i]]})
    else:
        elementDic[elementSeqList[i]].append(elementlocationList[i])
hairpinRagion = []
for d in elementDic.keys():
    if d[0] == 'h':
        hairpinRagion = hairpinRagion + elementDic[d]
if len(hairpinRagion) >= 1:
    apicalLoopSize = max([len(range(item[0], item[1]+1)) for item in hairpinRagion])
else:
    apicalLoopSize = 0
# Calculate the overlap of miRNA with the hairpin
# startPos and endPos are the locations of the miRNA in the corePrecusor. 0-based
startPos, endPos = getMiRNAPosition(miRNASeq, premiRNASeq)
endPos = endPos - 1
if len(hairpinRagion) >= 1:
    overlapLen = max([len(set(range(startPos, endPos+1)) & set(range(item[0], item[1]+1))) for item in hairpinRagion])
else:
    overlapLen = 0
armType = 'arm5'
if overlapLen == 0:
    if armType == 'arm5':
        distanceToLoop = min([item[0]-1-endPos for item in hairpinRagion])
    else:
        distanceToLoop = min([startPos-item[1]-1 for item in hairpinRagion])
else:
    for item in hairpinRagion:
        if len(set(range(startPos, endPos+1)) & set(range(item[0], item[1]+1))) == overlapLen:
            if armType == 'arm5':
                distanceToLoop = item[0]-1-endPos
            else:
                distanceToLoop = startPos - item[1] - 1
            break
# armTypeOverlenDic.update({armType: [overlapLen, distanceToLoop]})
# if len(self.miRNASeq) == 2:
#     armType2 = checkArm(self.premiRNASeq, self.premiRNAStructure, self.miRNASeq[1])
#     startPos2, endPos2 = getMiRNAPosition(self.miRNASeq[1], self.premiRNASeq)
#     endPos2 = endPos2 - 1
#     if len(hairpinRagion) >= 1:
#         overlapLen2 = max([len(set(range(startPos2, endPos2+1)) & set(range(item[0], item[1]+1))) for item in hairpinRagion])
#     else:
#         overlapLen2 = 0
#     if overlapLen2 == 0:
#         if armType2 == 'arm5':
#             distanceToLoop2 = min([item[0]-1-endPos2 for item in hairpinRagion])
#         else:
#             distanceToLoop2 = min([startPos2-item[1]-1 for item in hairpinRagion])
#     else:
#         for item in hairpinRagion:
#             if len(set(range(startPos2, endPos2+1)) & set(range(item[0], item[1]+1))) == overlapLen2:
#                 if armType2 == 'arm5':
#                     distanceToLoop2 = item[0]-1-endPos2
#                 else:
#                     distanceToLoop2 = startPos2 - item[1]-1
#                 break
#     armTypeOverlenDic.update({armType2: [overlapLen2, distanceToLoop2]})
# Calculate the stem length of precusor.
stemLen = 0
for d in elementDic.keys():
    if d[0] == 's':
        stemLen = stemLen + max([item[1]+1-item[0] for item in elementDic[d]])
# Check whether apical UGU/UGUG motif exist in hairpin region
flag = 'No'
for d in elementDic.keys():
    if d[0] == 'h':
        for item in elementDic[d]:
            seqTmp = self.premiRNASeq[item[0]:item[1]]
            if 'UGU' in seqTmp or 'UGUG' in seqTmp:
                flag = 'Yes'
                break
# pair state
if len(self.miRNASeq) == 2:
    pairState = 'Yes'
else:
    pairState = 'No'
    
return (hairpinCount, bindingCount, interiorLoopCount, armType, apicalLoopSize, armTypeOverlenDic, stemLen, flag, self.mfe, pairState)

NameError: name 'miRNASeq' is not defined

In [24]:
def to_element_string(bulgeGraphInstance, with_numbers=False):
	output_str = [' '] * (bulgeGraphInstance.seq_length + 1)
	output_nr = [' '] * (bulgeGraphInstance.seq_length + 1)
	for d in bulgeGraphInstance.defines.keys():
		for resi in bulgeGraphInstance.define_residue_num_iterator(d, adjacent=False):
			output_str[resi] = d[0]
			output_nr[resi] = d[-1]
	if with_numbers:
		return "".join(output_str).strip()+"\n"+"".join(output_nr).strip()
	else:
		return "".join(output_str).strip()

In [27]:
def getMiRNAPosition(miRNASeq, RNASequence):
	startPos = RNASequence.find(miRNASeq)
	endPos = startPos + len(miRNASeq)
	return [startPos, endPos]

#get miRNA structure
def getMiRNAStructure(miRNASeq, RNASequence, RNAStructure):
	[startPos, endPos] = getMiRNAPosition(miRNASeq, RNASequence)
	mirnaStruct = RNAStructure[startPos:endPos]
	return mirnaStruct


# Here starts our feature-generating code. 

In [30]:
region = 'chrX:50008481_50008504'

In [76]:
def lookup_in_hg38(region, hg38):    
    chr_ = re.search('chr[^:]*', region).group(0)
    start = re.search('(?<=:)[0-9]*(?=_)', region).group(0)
    end = re.search('(?<=_)[0-9]*$', region).group(0)
    try:
        miRNA_seq = re.sub('T', 'U', hg38.fetch(region=re.sub('_', '-', region)))
    except:
        return [region] + [None] * 6
    precursor_1_region = chr_ + ':' + str(int(start) - 70) + '-' + str(int(end) + 20)
    precursor_2_region = chr_ + ':' + str(int(start) - 20) + '-' + str(int(end) + 70)
    precursor_1 = re.sub('T', 'U', hg38.fetch(region=precursor_1_region))
    precursor_2 = re.sub('T', 'U', hg38.fetch(region=precursor_2_region))
    return np.array([region, miRNA_seq, precursor_1, precursor_2, chr_, start, end])

In [50]:
lookup_in_hg38(region, hg38=hg38)

array(['AAUGCACCUGGGCAAGGAUUCUGA',
       'UUCACAGGGCUUUGUGUUCUGCUCCCCCUCUCUAAUCCUUGCUACCUGGGUGAGAGUGCUGUCUGAAUGCAAUGCACCUGGGCAAGGAUUCUGAGAGCGAGAGCUUCAUCUUCG',
       'UGAGAGUGCUGUCUGAAUGCAAUGCACCUGGGCAAGGAUUCUGAGAGCGAGAGCUUCAUCUUCGUGUAGGACAGAGCGUCUUUCUGUCCUCCUAUCAUCCAUUCCACAAGUAUU',
       'chrX', '50008481', '50008504'], dtype='<U114')

In [51]:
df = pd.read_csv('SVM_training/all_tissues_mapped_dataset_15_refined_miRGeneDB_total_features_updated.csv')

In [61]:
df['region'] = [re.search('chr.*', cn).group(0)[:-1] for cn in df['clusterName']]

In [77]:
sequences = []
for i, region in enumerate(df['region']):
    sequences.append(lookup_in_hg38(region, hg38))
seq_df = pd.DataFrame(np.array(sequences), columns=['region', 'miRNA_seq', 'precursor1_seq', 'precursor2_seq', 'chr', 'start', 'end'])

In [86]:
set(seq_df['region'].values == df['region'].values)

{True}

In [107]:
df.head()

,realMicRNA,realMicRNAName,clusterName,seqCount,readCountSum,exactMatchRatio,headUnstableLength,tailUnstableLength,head_minus3_templateNucleotide,head_minus3_TemplateNucleotide_percentage,...,percentage_PairedInMiRNA,hairpin_count,binding_count,interiorLoopCount,apicalLoop_size,stem_length,mFE,count_bindings_in_miRNA,UGU_UGUG_motif,pair_state
0,1,hsa-miR-200b-5p,mapped_mirna_ERR038410:miRCluster_1_22:chr1:11...,3,11,0.636364,0,2,G,0.0,...,0.95,1,34,6,6,34,-42.9,20,No,Yes
1,1,hsa-miR-200b-3p,mapped_mirna_ERR038410:miRCluster_2_24:chr1:11...,20,159,0.327044,2,4,C,0.0,...,0.90,1,32,5,6,32,-40.9,18,No,Yes
2,1,hsa-miR-200a-3p,mapped_mirna_ERR038410:miRCluster_3_23:chr1:11...,15,235,0.565957,0,1,C,0.0,...,0.78,1,32,4,5,32,-45.5,18,No,No
3,1,hsa-miR-429,mapped_mirna_ERR038410:miRCluster_4_22:chr1:11...,3,27,0.925926,0,0,G,0.0,...,0.77,1,29,3,10,29,-38.8,17,No,No
4,1,hsa-miR-34a-5p,mapped_mirna_ERR038410:miRCluster_31_23:chr1:9...,10,70,0.828571,0,4,C,0.0,...,0.90,1,35,5,4,35,-39.2,18,No,No


In [109]:
df[['miRNA_seq', 'precursor1_seq', 'precursor2_seq', 'chr', 'start', 'end']] = seq_df[['miRNA_seq', 'precursor1_seq', 'precursor2_seq', 'chr', 'start', 'end']]

In [ ]:
f1 = os.path.join(self.tmpdir, 'prunedPrecusor.fa')
f2 = os.path.join(self.tmpdir, 'prunedPrecusor.str')
os.system('%s < %s --noPS --noLP > %s'%(self.rnafoldCmdTmp, f1, f2))